In [1]:
# %conda install -c conda-forge ta-lib -q
# %pip install -r requirements.txt -q

# import boto3
import data
import glob
import matplotlib.pyplot as plt
import models
import numpy as np
import os
import pandas as pd
import pypickle
import strategy
import time

from data import * 
from feature_engineering import FeaturesEngineering
from models import Models
from io import StringIO
from IPython.display import display
from sklearn.linear_model import LassoCV
from utils import log

2025-02-26 21:59:23.858925: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-26 21:59:23.869048: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1740578363.881284   73923 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1740578363.885045   73923 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-02-26 21:59:23.897617: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
pypickle_overwrite = True

models = Models(use_xgboost_gpu=True)

# Initializing the FeaturesEngineering obj
feat = FeaturesEngineering()
# Default config
feat.default_config

dict_data = {}
dict_cv_features = {}

In [3]:
dir_project = os.getcwd()
dir_models = os.path.join(dir_project, 'models')
dir_models_base = os.path.join(dir_project, 'models', 'base')
dir_models_scaler = os.path.join(dir_project, 'models', 'scaler')
dir_models_best = os.path.join(dir_project, 'models', 'best')

print(f"""
Project directory: {dir_project}
Models directory: {dir_models}
Base models directory: {dir_models_base}
Scalers directory: {dir_models_scaler}
Best models directory: {dir_models_best}
""")


Project directory: /home/alfred/projects/mscfe-masters-project/src
Models directory: /home/alfred/projects/mscfe-masters-project/src/models
Base models directory: /home/alfred/projects/mscfe-masters-project/src/models/base
Scalers directory: /home/alfred/projects/mscfe-masters-project/src/models/scaler
Best models directory: /home/alfred/projects/mscfe-masters-project/src/models/best



# Read raw data

In [4]:
# Define S3 bucket and file path
bucket_name = 'mscfe-capstone-project-bucket'
file_key = 'filtered_data.csv'

filtered = pd.read_csv('filtered_data.csv').drop(['Unnamed: 0'], axis=1)
display(filtered)

,seriesX,seriesY,correlation,spread_adf_stat,spread_adf_pValue,spread_stationary,series_coint_stat,series_coint_pValue,series_cointegrated
0,GOOG,GOOGL,0.999931,-4.971937,0.000025,True,-4.976339,0.000182,True
1,META,DAXEX_ETF,0.888934,-2.917056,0.043400,True,-3.483091,0.033924,True
2,AMZN,BNBUSDT,0.877059,-3.631203,0.005192,True,-3.545100,0.028601,True
3,AMZN,TRXUSDT,0.814577,-3.869803,0.002269,True,-3.387413,0.043768,True
4,SPY_ETF,AVAXUSDT,0.847292,-3.309927,0.014441,True,-3.698512,0.018406,True
5,SPY_ETF,ETHUSDT,0.862483,-3.428205,0.010034,True,-3.737367,0.016394,True
6,EQQQ_ETF,BNBUSDT,0.886060,-3.816404,0.002744,True,-3.670509,0.019987,True
7,BNBUSDT,ADAUSDT,0.901459,-3.279504,0.015820,True,-3.775562,0.014607,True
8,BNBUSDT,BTCUSDT,0.884604,-2.909961,0.044202,True,-3.393160,0.043116,True
9,BNBUSDT,TRXUSDT,0.933562,-3.656380,0.004770,True,-4.003675,0.007095,True


In [5]:
print('********************')
pairs = filtered['seriesX'].iloc[1:] + ' ' + filtered['seriesY'].iloc[1:]
print('Trading pairs:')
for p in pairs:
    print(p)
print('********************')

unique_tickers = pd.concat([filtered['seriesX'].iloc[1:], filtered['seriesY'].iloc[1:]]).drop_duplicates()
print('Unique tickers:')
for ut in unique_tickers:
    print(ut)
print('********************')

# Preview the source data
file_key = 'df_full.csv'
df_full = pd.read_csv('df_full.csv').drop(['Unnamed: 0'], axis=1)
# obj = s3.get_object(Bucket=bucket_name, Key=file_key)
# df_full = pd.read_csv(StringIO(obj['Body'].read().decode('utf-8'))).drop(['Unnamed: 0'], axis=1)
display(df_full.sample(5))

********************
Trading pairs:
META DAXEX_ETF
AMZN BNBUSDT
AMZN TRXUSDT
SPY_ETF AVAXUSDT
SPY_ETF ETHUSDT
EQQQ_ETF BNBUSDT
BNBUSDT ADAUSDT
BNBUSDT BTCUSDT
BNBUSDT TRXUSDT
BNBUSDT XRPUSDT
********************
Unique tickers:
META
AMZN
SPY_ETF
EQQQ_ETF
BNBUSDT
DAXEX_ETF
TRXUSDT
AVAXUSDT
ETHUSDT
ADAUSDT
BTCUSDT
XRPUSDT
********************


,OpenTime,Open,High,Low,Close,Symbol
116019,2024-10-16 11:40:00,67795.9900,67807.2600,67643.2400,67721.0200,BTCUSDT
120446,2024-10-18 00:35:00,0.3478,0.3479,0.3472,0.3474,ADAUSDT
355898,2025-01-07 22:35:00,701.7900,702.0800,701.1200,701.1300,BNBUSDT
14946,2024-11-01 07:10:00,793.2000,793.2000,793.1000,793.1000,FTSE_ETF
228387,2024-11-24 12:05:00,40.1700,40.2900,40.1600,40.1900,AVAXUSDT


# Features Engineering

## Create and normalizing features

1. Create features from pairs data
2. Load saved scaler
3. Perform normalization / standardization

In [6]:
scaler_type = "StandardScaler"
dict_data['df_feat'] = {}
dict_data['df_norm'] = {}
dict_data['scaler'] = {}
i = 1

for p in pairs:

    print(f"({i}) Pairs: {p}")
    tickerX, tickerY = p.split(' ')
    
    # 1. Create features from pairs data
    df_feat = feat.create_features(
        tickerX, 
        tickerY,
        df_full[df_full['Symbol']==tickerX],
        df_full[df_full['Symbol']==tickerY],
        config=None, # using default config in the FeaturesEngineering
        dropna=True
    )

    # 2. Load saved scaler
    scaler_name = f"{scaler_type.lower()}_{p.replace(' ', '_')}.pkl"
    dir_scaler = os.path.join(dir_models_scaler, scaler_name)
    scaler = pypickle.load(dir_scaler)

    # 3. Perform normalization / standardization
    data_scaled = scaler.transform(df_feat)
    df_norm = pd.DataFrame(
        data_scaled, index=df_feat.index, columns=df_feat.columns
    )
    df_norm = df_norm.rename(columns={'Spread':'NormalizedSpread'})
    
    dict_data['df_feat'][p] = df_feat
    dict_data['df_norm'][p] = df_norm
    dict_data['scaler'][p] = scaler

    i+=1
    print('')

(1) Pairs: META DAXEX_ETF
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_META_DAXEX_ETF.pkl]

(2) Pairs: AMZN BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_BNBUSDT.pkl]

(3) Pairs: AMZN TRXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_AMZN_TRXUSDT.pkl]

(4) Pairs: SPY_ETF AVAXUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_SPY_ETF_ETHUSDT.pkl]

(6) Pairs: EQQQ_ETF BNBUSDT
[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/scaler/standardscaler_EQQQ_ETF_BNBUSDT.pkl]

(7) Pairs: BNBUSDT ADAUSDT
[pypickle] Pickle file loaded: [/ho

## Read LassoCV selected features

In [7]:
lasso_cv_name = f"lasso_cv_features.pkl"
dir_lasso = os.path.join(dir_models, lasso_cv_name)
dict_cv_features = pypickle.load(dir_lasso)

for key in dict_cv_features:
    print(f"{key}: {dict_cv_features[key]}")

[pypickle] Pickle file loaded: [/home/alfred/projects/mscfe-masters-project/src/models/lasso_cv_features.pkl]
META DAXEX_ETF: ['META_EMA8', 'META_EMA55', 'META_RSI14', 'META_ATR14', 'META_StochD3', 'META_CCI20', 'META_WILLR14', 'DAXEX_ETF_EMA8', 'DAXEX_ETF_EMA21', 'DAXEX_ETF_MACD', 'DAXEX_ETF_BBupper', 'DAXEX_ETF_BBmiddle', 'DAXEX_ETF_BBlower', 'DAXEX_ETF_StochD3']
AMZN BNBUSDT: ['AMZN_EMA8', 'AMZN_EMA21', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_WILLR14', 'BNBUSDT_EMA8', 'BNBUSDT_EMA55', 'BNBUSDT_MACD', 'BNBUSDT_RSI14', 'BNBUSDT_ATR14', 'BNBUSDT_StochD3', 'BNBUSDT_CCI20', 'BNBUSDT_WILLR14']
AMZN TRXUSDT: ['AMZN_EMA8', 'AMZN_EMA55', 'AMZN_RSI14', 'AMZN_ATR14', 'AMZN_StochD3', 'AMZN_CCI20', 'AMZN_WILLR14', 'TRXUSDT_MACD']
SPY_ETF AVAXUSDT: ['SPY_ETF_EMA8', 'SPY_ETF_EMA55', 'SPY_ETF_MACD', 'SPY_ETF_RSI14', 'SPY_ETF_BBupper', 'SPY_ETF_BBmiddle', 'SPY_ETF_BBlower', 'SPY_ETF_ATR14', 'SPY_ETF_StochK14', 'SPY_ETF_StochD3', 'SPY_ETF_CCI20', 'SPY_ETF_WILLR14', 'AVAXUSDT_E

# Training the Base Models

In [8]:
dict_base = {
    'ridge': models.ridge_regression,
    'xgb': models.xgboost_regression,
    'rnn': models.rnn_regression,
    'lstm': models.lstm_regression,
    'transformer': models.transformer_regression,
}

In [12]:
i = 1
verbose = False
list_results = []

for p in pairs:
    print(f"({i}) Pairs: {p}")

    # 1. Read normalized features
    df_norm = dict_data['df_norm'][p]
    
    # 2. Create X, y based on LassoCV
    lasso_features_cols = dict_cv_features[p]
    target_col = df_norm.columns.tolist()[-1]
    df_lasso = pd.concat([df_norm[lasso_features_cols], df_norm[target_col]], axis=1)

    for algo in dict_base.keys():
        dir_base = os.path.join(dir_models_base, algo)
        
        model_algo = dict_base[algo]
        base_model, mse, df_test, time, memory = model_algo(df=df_lasso, p=p, verbose=verbose)

        print(f"{algo}, MSE: {mse:.6f}, data_points: {len(df_lasso):,}, time_taken: {time:.4f} sec.")

        # 4. Save best model future use like testing and strategy
        model_name = f"base_{algo}_{p.replace(' ', '_')}.pkl"
        dir_model_name = os.path.join(dir_base, model_name)
        pypickle.save(dir_model_name, base_model, overwrite=pypickle_overwrite)

        list_results.append({
            'pair':p,
            'algorithm': algo,
            'base_mse': mse,
            'data_points': len(df_lasso),
            'features': len(df_lasso.columns),
            'time_taken': time,
            'memory_used': memory
        })

    i+=1
    print('')

dir_param = os.path.join(dir_models_base, 'base_model_results.csv')
df_base_results = pd.DataFrame(list_results)
df_base_results.to_csv(dir_param, index=False)
display(df_base_results)

(1) Pairs: META DAXEX_ETF
ridge, MSE: 0.002758, data_points: 1,145, time_taken: 0.0018 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/ridge/base_ridge_META_DAXEX_ETF.pkl]
xgb, MSE: 0.035702, data_points: 1,145, time_taken: 0.1713 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/xgb/base_xgb_META_DAXEX_ETF.pkl]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step 
rnn, MSE: 0.028946, data_points: 1,145, time_taken: 5.6794 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/rnn/base_rnn_META_DAXEX_ETF.pkl]
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
lstm, MSE: 0.041714, data_points: 1,145, time_taken: 7.5416 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/lstm/base_lstm_META_DAXEX_ETF.pkl]
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 102ms/step
transformer, MSE: 0.052909, data_points: 1,145, time_taken: 10.3638 sec.
[pypickle] Pickle file s

2025-02-26 22:09:56.119193: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 4 bytes spill stores, 4 bytes spill loads

2025-02-26 22:09:56.173282: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 4 bytes spill stores, 4 bytes spill loads

2025-02-26 22:09:56.195422: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_28', 8 bytes spill stores, 8 bytes spill loads

2025-02-26 22:09:56.385876: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_8', 8 bytes spill stores, 8 bytes spill loads

2025-02-26 22:09:56.567734: I external/local_xla/xla/stream_executor/cuda/cud

32/32 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step  
transformer, MSE: 0.068668, data_points: 4,997, time_taken: 16.8997 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/transformer/base_transformer_SPY_ETF_AVAXUSDT.pkl]

(5) Pairs: SPY_ETF ETHUSDT
ridge, MSE: 0.000894, data_points: 4,997, time_taken: 0.0015 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/ridge/base_ridge_SPY_ETF_ETHUSDT.pkl]
xgb, MSE: 0.001257, data_points: 4,997, time_taken: 0.0566 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/xgb/base_xgb_SPY_ETF_ETHUSDT.pkl]
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step  
rnn, MSE: 0.002427, data_points: 4,997, time_taken: 16.4106 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/rnn/base_rnn_SPY_ETF_ETHUSDT.pkl]
32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
lstm, MSE: 0.002092, data_points: 4,997, time_taken: 33.1289

2025-02-26 22:11:56.808199: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:397] ptxas warning : Registers are spilled to local memory in function 'input_reduce_select_fusion', 16 bytes spill stores, 12 bytes spill loads
ptxas warning : Registers are spilled to local memory in function 'input_add_reduce_select_fusion', 12 bytes spill stores, 8 bytes spill loads



21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 42ms/step 
transformer, MSE: 0.031880, data_points: 3,217, time_taken: 14.5685 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/transformer/base_transformer_EQQQ_ETF_BNBUSDT.pkl]

(7) Pairs: BNBUSDT ADAUSDT
ridge, MSE: 0.000150, data_points: 31,051, time_taken: 0.0028 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/ridge/base_ridge_BNBUSDT_ADAUSDT.pkl]
xgb, MSE: 0.000225, data_points: 31,051, time_taken: 0.0594 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/xgb/base_xgb_BNBUSDT_ADAUSDT.pkl]
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 624us/step 
rnn, MSE: 0.002679, data_points: 31,051, time_taken: 99.8963 sec.
[pypickle] Pickle file saved: [/home/alfred/projects/mscfe-masters-project/src/models/base/rnn/base_rnn_BNBUSDT_ADAUSDT.pkl]
194/194 ━━━━━━━━━━━━━━━━━━━━ 0s 563us/step
lstm, MSE: 0.001019, data_points: 31,051, time_taken

,pair,algorithm,base_mse,data_points,features,time_taken,memory_used
0,META DAXEX_ETF,ridge,0.002758,1145,15,0.001789,0.000000
1,META DAXEX_ETF,xgb,0.035702,1145,15,0.171267,0.000000
2,META DAXEX_ETF,rnn,0.028946,1145,15,5.679418,-8.207031
3,META DAXEX_ETF,lstm,0.041714,1145,15,7.541587,15.050781
4,META DAXEX_ETF,transformer,0.052909,1145,15,10.363810,289.265625
5,AMZN BNBUSDT,ridge,0.001193,4991,16,0.006104,0.000000
6,AMZN BNBUSDT,xgb,0.006104,4991,16,0.062013,1.000000
7,AMZN BNBUSDT,rnn,0.003206,4991,16,17.041029,45.390625
8,AMZN BNBUSDT,lstm,0.003938,4991,16,33.034864,6.636719
9,AMZN BNBUSDT,transformer,0.018339,4991,16,16.437440,193.308594
